In [ ]:
# essentials
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import datetime
from pathlib import Path

# tensorflow tools
from keras.preprocessing import image
from keras.applications import imagenet_utils
from keras import Model
from keras.layers import Dense, Input, Conv2D, MaxPooling2D, Concatenate, Add, Activation, Dropout, GlobalAveragePooling2D
from keras.callbacks import ModelCheckpoint, EarlyStopping, TensorBoard
from keras.utils import plot_model

In [ ]:
# Filepaths to training and test sets
base_dir = './data/Fruits-360/fruits-360_dataset/fruits-360'

train_filepaths = list(Path(base_dir + '/Training').glob(r'**/*.jpg'))
test_filepaths = list(Path(base_dir + '/Test').glob(r'**/*.jpg'))

In [ ]:
# Obtaining labels from filepaths function
def get_fruit_label(filepaths):
    labels = [str(filepath).split('\\')[-2].split(' ')[0] for filepath in filepaths]

    filepaths = pd.Series(filepaths, name='Filepath').astype(str)
    labels = pd.Series(labels, name='Label')

    fruits_df = pd.concat([filepaths, labels], axis=1)

    return fruits_df

In [ ]:
# Creating datasets with image filepaths and corresponding labels
train_df = get_fruit_label(train_filepaths)
test_df = get_fruit_label(test_filepaths)

In [ ]:
# Getting a list of labels
label_list = train_df.Label.unique().tolist()

In [ ]:
# Fixed parameters
image_size = (224, 224, 3)
batch_size = 32
num_classes = len(label_list)

In [ ]:
# Creating data generators for training and test sets
train_datagen = image.ImageDataGenerator(
    validation_split=0.2,
    rotation_range=180,
    shear_range=10,
    horizontal_flip=True,
    vertical_flip=True
)

test_datagen = image.ImageDataGenerator()

In [ ]:
train_gen = train_datagen.flow_from_dataframe(
    dataframe=train_df,
    x_col='Filepath',
    y_col='Label',
    batch_size=batch_size,
    class_mode='categorical',
    subset='training',
    shuffle=True,
    target_size=image_size[:-1]
)

validation_gen = train_datagen.flow_from_dataframe(
    dataframe=train_df,
    x_col='Filepath',
    y_col='Label',
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation',
    shuffle=True,
    target_size=image_size[:-1]
)

test_gen = test_datagen.flow_from_dataframe(
    dataframe=test_df,
    x_col='Filepath',
    y_col='Label',
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=True,
    target_size=image_size[:-1]
)

<h1> Custom model </h1>

In [ ]:
# Custom preprocess function
def custom_preprocess(input_image):
    return imagenet_utils.preprocess_input(input_image, mode='tf')

In [ ]:
# Applying custom preprocess function on train and test data generators
train_datagen.preprocessing_function = custom_preprocess
test_datagen.preprocessing_function = custom_preprocess

In [ ]:
# Custom Inception-Residual block
def custom_block(input_layer, filters, name=None):
    padding = 'same'
    f1, f2, f3, f4 = filters

    input_shortcut = input_layer

    branch0 = Conv2D(filters=f1, kernel_size=(1, 1) , strides=(1, 1), padding=padding, activation='relu', name=name+'_b0')(input_layer)
    branch1 = Conv2D(filters=f2, kernel_size=(1, 1), strides=(1, 1), padding=padding, activation='relu', name=name+'_b1_0')(input_layer)
    branch1 = Conv2D(filters=f3, kernel_size=(3, 3), strides=(1, 1), padding=padding, activation='relu', name=name+'_b1_1')(branch1)

    mixed = Concatenate(axis=3, name=name+'_concat')([branch0, branch1])
    filexp = Conv2D(filters=f4, kernel_size=(1, 1), strides=(1, 1), padding=padding, name=name+'_filexp')(mixed)

    output = Add()([input_shortcut, filexp])
    output = Activation('relu')(output)

    return output

In [ ]:
# Custom model architecture
def custom_model(n_classes, input_shape=(224, 224, 3)):

    x_input = Input(input_shape)

    x = Conv2D(32, kernel_size=(3, 3), strides=(2, 2), padding='valid', activation='relu', name='conv1')(x_input)
    x = Conv2D(64, kernel_size=(3, 3), strides=(1, 1), padding='valid', activation='relu', name='conv2')(x)
    x = Conv2D(128, kernel_size=(3, 3), strides=(1, 1), padding='valid', activation='relu', name='conv3')(x)

    x = MaxPooling2D(pool_size=(3, 3), strides=(2, 2), padding='valid', name='maxpool1')(x)

    x = custom_block(x, [48, 32, 64, 128], 'block1')
    x = Conv2D(256, kernel_size=(3, 3), strides=(2, 2), padding='valid', activation='relu', name='conv4')(x)
    x = custom_block(x, [96, 64, 128, 256], 'block2')
    x = Conv2D(512, kernel_size=(3, 3), strides=(2, 2), padding='valid', activation='relu', name='conv5')(x)

    x = GlobalAveragePooling2D(data_format='channels_last')(x)
    x = Dropout(0.2, name='dropout1')(x)
    x = Dense(n_classes, activation='softmax', name='output')(x)

    model = Model(inputs=x_input, outputs=x)

    return model

In [ ]:
# Custom model construction and compilation
model_custom = custom_model(num_classes, image_size)
model_custom.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
# Plotting custom model's architecture
plot_model(model_custom, to_file='model.png', show_shapes=True, show_layer_names=True)

In [ ]:
# Creating callbacks
log_dir = "logs/Custom_model/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)
checkpoint_callback = ModelCheckpoint('custom_model.h5', monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
earlystopping_callback = EarlyStopping(monitor='val_loss', mode='min', patience=3, restore_best_weights=True)

callbacks = [tensorboard_callback, checkpoint_callback, earlystopping_callback]

In [ ]:
# Custom model learning
history_custom = model_custom.fit(train_gen, validation_data=validation_gen, epochs=25, callbacks=callbacks)

In [ ]:
# Loading of the best weights achieved
model_custom.load_weights('custom_model.h5')

In [ ]:
# Custom model evaluation
loss_custom, accuracy_custom = model_custom.evaluate(test_gen)

In [ ]:
# Plotting a sample of images with predicted and true labels
test_datagen.preprocessing_function = None
X_sample, y_sample = test_gen.next()

apply_preprocess = np.vectorize(custom_preprocess)
predictions_custom = model_custom.predict(apply_preprocess(X_sample), verbose=0)

predictions_custom = [label_list[np.argmax(prediction)] for prediction in predictions_custom]
true_labels_custom = [label_list[np.argmax(sample)] for sample in y_sample]

fig, axes = plt.subplots(4, 8, figsize=(15, 7), subplot_kw={'xticks' : [], 'yticks' : []})
for i, ax in enumerate(axes.flat):
    ax.imshow(X_sample[i]/255.)
    ax.set_title(f'True: {true_labels_custom[i]}\nPredicted: {predictions_custom[i]}', fontsize=8)
plt.tight_layout(pad=0.5)
plt.show()